In [3]:
import pandas as pd
df = pd.read_csv(r"data\000001SH_index.csv")
df

,id,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,0,000001.SH,19980105,1220.4730,1200.9480,1220.4950,1200.2220,1194.1020,26.3710,2.2084,4371532.0,5.363528e+06
1,1,000001.SH,19980106,1233.6200,1223.7300,1233.6350,1215.4090,1220.4730,13.1470,1.0772,5148664.0,6.196189e+06
2,2,000001.SH,19980107,1244.0700,1233.9560,1244.0980,1231.1460,1233.6200,10.4500,0.8471,5756557.0,6.276869e+06
3,3,000001.SH,19980108,1237.1640,1243.1210,1250.0050,1235.8850,1244.0700,-6.9060,-0.5551,4069472.0,4.535351e+06
4,4,000001.SH,19980109,1239.9010,1233.7010,1244.9990,1221.2970,1237.1640,2.7370,0.2212,5357096.0,6.395907e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
5425,5425,000001.SH,20200527,2836.8036,2847.3198,2848.9965,2831.9281,2846.5473,-9.7437,-0.3423,198149343.0,2.279344e+08
5426,5426,000001.SH,20200528,2846.2217,2838.2072,2861.9241,2820.1455,2836.8036,9.4181,0.3320,207546019.0,2.392343e+08
5427,5427,000001.SH,20200529,2852.3512,2835.5840,2855.3756,2829.6271,2846.2217,6.1295,0.2154,206792804.0,2.337990e+08
5428,5428,000001.SH,20200601,2915.4311,2871.9641,2917.1508,2871.9641,2852.3512,63.0799,2.2115,260196050.0,3.074162e+08


In [1]:
import torch
import torch.nn as nn

# 定义LSTM
lstm = nn.LSTM(input_size=8, hidden_size=32, num_layers=1, batch_first=True)

# 随机输入数据 (batch_size, seq_len, input_size)
input_data = torch.randn(4, 10, 8)

# 前向传播
out, (hidden, cell) = lstm(input_data)

# 输出形状
print(f"hidden.shape: {hidden.shape}")
print(f"hidden[-1].shape: {hidden[-1].shape}")


hidden.shape: torch.Size([1, 4, 32])
hidden[-1].shape: torch.Size([4, 32])


In [ ]:
df

In [ ]:
from LSTMModel import lstm # 导入自定义的LSTM模型类
from dataset import getData  # 导入数据处理函数
from parser_my import args # 导入命令行参数解析模块
import torch


def eval():
    # 加载预训练模型
    # model = torch.load(args.save_file)
    model = lstm(input_size=args.input_size, hidden_size=args.hidden_size, num_layers=args.layers , output_size=1)  # 创建LSTM模型实例
    model.to(args.device)  # 将模型移到指定的设备（CPU或GPU）
    checkpoint = torch.load(args.save_file)  # 加载模型的状态字典
    model.load_state_dict(checkpoint['state_dict']) # 将状态字典加载到模型中
    preds = [] # 初始化预测值列表
    labels = []  # 初始化真实标签值列表
    close_max, close_min, train_loader, test_loader = getData(args.corpusFile, args.sequence_length, args.batch_size)  # 获取训练和测试数据
    for idx, (x, label) in enumerate(test_loader):   # 遍历测试数据进行预测
        if args.useGPU:
            x = x.squeeze(1).cuda()  # batch_size,seq_len,input_size # 如果使用GPU，将数据移到GPU并调整维度
        else:
            x = x.squeeze(1)  # 调整数据维度
        pred = model(x)  # 使用模型进行预测
        print(pred.shape)
        list = pred.data.squeeze(1).tolist() # 将预测结果转换为列表
        preds.extend(list[-1])  # 记录每个样本的最后一个预测值
        # 将预测结果转换为列表，然后将其包装在列表中，再将其添加到 preds 中
        # pred_list = [pred.data.squeeze(1).tolist()] 
        # preds.extend(pred_list[-1])  # 记录每个样本的最后一个预测值

        labels.extend(label.tolist())  # 记录真实标签值
     # 打印预测值和真实值
    for i in range(len(preds)):
        # 将预测值反标准化并打印、将真实值反标准化并打印
        print('预测值是%.2f,真实值是%.2f' % (
        preds[i][0] * (close_max - close_min) + close_min, labels[i] * (close_max - close_min) + close_min))  

eval()